# Benchmarking BERT with ONNX Optimizations

In [ ]:
pip install --upgrade --upgrade-strategy eager "optimum[onnxruntime]"

## Apply ONNX Optimization and export to ONNX

In [2]:
from optimum.onnxruntime import (
    AutoOptimizationConfig,
    ORTOptimizer,
    ORTModelForSequenceClassification,
)

model_id = "nlptown/bert-base-multilingual-uncased-sentiment"
save_dir = "onnx"

# Load a PyTorch model and export it to the ONNX format
model = ORTModelForSequenceClassification.from_pretrained(model_id, export=True)

# Create the optimizer
optimizer = ORTOptimizer.from_pretrained(model)

# Define the optimization strategy by creating the appropriate configuration
optimization_config = AutoOptimizationConfig.O2()

# Optimize the model
optimizer.optimize(save_dir=save_dir, optimization_config=optimization_config)

/Users/andrewreed/Documents/hf-notebooks/bert-onnx-optimum-benchmark/.venv/lib/python3.10/site-packages/optimum/onnxruntime/configuration.py:779: FutureWarning: disable_embed_layer_norm will be deprecated soon, use disable_embed_layer_norm_fusion instead, disable_embed_layer_norm_fusion is set to True.
  warnings.warn(
Optimizing model...
Configuration saved in onnx/ort_config.json
Optimized model saved at: onnx (external data format: False; saved all tensor to one file: True)


PosixPath('onnx')

## Compare ONNX Optimized Model to PyTorch Model

### Load models & pipelines

In [48]:
from transformers import pipeline, AutoTokenizer, AutoModelForSequenceClassification

# load the pytorch model into a pipeline
tokenizer = AutoTokenizer.from_pretrained(model_id)
pytorch_model = AutoModelForSequenceClassification.from_pretrained(model_id)
pytorch_pipeline = pipeline(
    "text-classification", model=pytorch_model, tokenizer=tokenizer
)

# load the optimized onnx model into a pipeline
onnx_o2_model = ORTModelForSequenceClassification.from_pretrained(
    save_dir, file_name="model_optimized.onnx"
)
onnx_o2_pipeline = pipeline(
    "text-classification", model=onnx_o2_model, tokenizer=tokenizer
)

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.
Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


### Load a dummy dataset

In [52]:
from datasets import load_dataset

ds = load_dataset("tyqiangz/multilingual-sentiments", "all")
sample = ds["train"].select(range(2000))

In [53]:
sample

Dataset({
    features: ['text', 'source', 'language', 'label'],
    num_rows: 2000
})

In [56]:
from time import perf_counter
import numpy as np


def measure_latency(pipe, sample):
    latencies = []

    # warm up
    for _ in range(10):
        _ = pipe(sample[0]["text"])

    # benchmark
    for i in range(len(sample)):

        start_time = perf_counter()
        _ = pipe(sample[i]["text"])
        latency = perf_counter() - start_time

        latencies.append(latency)

    print("Number of samples run:", len(latencies))

    # Compute run statistics
    time_avg_ms = 1000 * np.mean(latencies)
    time_std_ms = 1000 * np.std(latencies)

    return f"Average latency (ms) - {time_avg_ms:.2f} +\- {time_std_ms:.2f}"

In [57]:
print("Running PyTorch model...")
print(f"Vanilla model: {measure_latency(pytorch_pipeline, sample)}")
print("\nRunning ONNX model...")
print(f"O2 Optimized: {measure_latency(onnx_o2_pipeline, sample)}")

Running PyTorch model...
Number of samples run: 2000
Vanilla model: Average latency (ms) - 29.87 +\- 5.30

Running ONNX model...
Number of samples run: 2000
O2 Optimized: Average latency (ms) - 14.81 +\- 6.55
